## Read the features. Eliminate those with correlation greater than 0.7
## Construct minimum spanning tree.

In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt

In [3]:
import networkx as nx
import pygsp

In [4]:
cd '/home/raghuram/Desktop/radiomics/TEXTURES/results'

/home/raghuram/Desktop/radiomics/TEXTURES/results


In [5]:
home = '/home/raghuram/Desktop/radiomics/TEXTURES/results'

In [6]:
features_folder = '/home/raghuram/Desktop/radiomics/TEXTURES'

In [7]:
folder_list = os.listdir(os.getcwd())

In [8]:
csv_file_dict = {}

In [9]:
def get_csv_files():
    for folder in folder_list:
        csv_file_list = sorted(os.listdir(os.path.join(home, folder, 'linear_regression', 'without_idh1')), key=lambda x: int(x.split('_')[0]))
        csv_file_dict[folder] = csv_file_list
    
    return csv_file_dict

In [10]:
modality_csv_files = get_csv_files()

In [11]:
df = pd.read_csv(os.path.join(home, 't2f', 'linear_regression', 'without_idh1', modality_csv_files['t2f'][0]))

In [12]:
features = list(df['response_variable'].unique())

In [13]:
alpha = 0.05
m = len(features)*5
bonferroni_correction_factor = alpha/(m)

In [14]:
# TODO: Make this function faster
def filter_relevant_features():
    results_dict = {}
    for modality in modality_csv_files.keys():
        results_dict[modality] = {}
        csv_file_list = modality_csv_files[modality]
        for csv_file in csv_file_list:
            df = pd.read_csv(os.path.join(home, modality, 'linear_regression', 'without_idh1', csv_file))
            experiment_number = csv_file.split('_')[0]   
            results_dict[modality][experiment_number] = ['IDH1', '1p_19q_co_del_status']
            for idx, feature in enumerate(features):
                feature_df = df[df['response_variable'] == feature]
    #             print(feature_df.head())
                if np.all(feature_df['pvals']>bonferroni_correction_factor):
                    results_dict[modality][experiment_number].append(feature)
                    continue
                else:
                    continue
    return results_dict

In [15]:
results_dict = filter_relevant_features()

In [16]:
def load_relevant_features(modality, experiment_number):
    try:
        assert(1<=experiment_number<=25)
        relevant_features = results_dict[modality][str(experiment_number)]
     
        df = pd.read_csv(os.path.join(features_folder, 'expt_'+modality+'.csv'))
        df = df[df['experiment_number'] == experiment_number]
        required_columns = results_dict[modality][str(experiment_number)]
        df = df[df.columns[df.columns.isin(required_columns)]]
        return df
    except AssertionError as e:
        print('Experiment number cannot be greater than twenty-five or less than one')

In [17]:
def get_features_to_exclude(df):
    K = np.array(df.corr())
    x, y = np.where(np.abs(K)>=0.7)
    correlated_pair = []
    drop_columns = []
    for value in zip(x,y):
        if value[0] == value[1]:
            continue
        if (value[1], value[0]) in correlated_pair:
            continue
        correlated_pair.append(value)
        drop_columns.append(value[1])
    
    return drop_columns, correlated_pair

In [18]:
def drop_correlated_features(df, drop_columns):
    dropped_feature_names = []
    for index in drop_columns:
        dropped_feature_names.append(df.columns[index])
    dropped_df = df.drop(columns=dropped_feature_names, inplace=False)
    return dropped_df

In [19]:
def form_minimum_spanning_tree(filtered_df):
    A = np.array(filtered_df.corr())
    G = nx.from_numpy_matrix(A)
    # From this graph form the minimum spanning tree
    K = nx.minimum_spanning_tree(G)
    
    return K

In [20]:
# Normalize the features within group to have unit norm
# Use five-fold cross-validation to see if the results make sense
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [21]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score

In [22]:
kfold = StratifiedKFold(n_splits=3, random_state=2019)
# classifier = SVC(kernel='rbf', probability=True,
#                    random_state=2019)
classifier = LogisticRegression(random_state=2019)

In [23]:
from sklearn.utils import shuffle

In [24]:
prediction_results_df = {}
prediction_results_normalized_df = {}
prediction_results_with_tv = {}

In [25]:
modality_list = ['t2f']

In [26]:
def save_plot(modality, fname, signal):
    plt.clf()
    plt.stem(np.linspace(1, signal.shape[0], num=signal.shape[0]), signal.T)
    plt.xlabel('Node index')
    plt.ylabel('Average graph signal for IDH-non-mut in the vertex domain.' )
    plt.savefig(os.path.join(features_folder, 'graph_fourier_transform', modality, fname))

In [29]:
from pygsp import plotting            

In [227]:
def form_graph(graph_signal):
        A = np.array(filtered_df.corr())
        B = nx.from_numpy_matrix(A)
        # From this graph form the minimum spanning tree
        M = nx.minimum_spanning_tree(B)
        
        # Obtain the adjacency matrix of the minimum spanning tree
        K = np.abs(nx.adjacency_matrix(B))
        
        G = pygsp.graphs.Graph(K.todense())
        return G

In [228]:
def normalize_signal(graph_signal):
    scaler = StandardScaler()
    return scaler.fit_transform(graph_signal)
    

In [225]:
def graph_fourier_bases(G):
    G.compute_fourier_basis()
    U, e = G.U, G.e
    
    return U, e

In [346]:
def compute_graph_gft(U, graph_signal):
    XG =  np.matmul(np.linalg.inv(U), graph_signal)
    return XG.T

In [454]:
def compute_graph_tv(G, graph_signal):
    tv = []
    G.compute_fourier_basis()
    L = G.L.todense()
    for idx, signal in enumerate(graph_signal):
        
        tv.append(np.dot(np.dot(signal.T, L), signal))
    
    return np.array(tv)
        

In [449]:
# Reference: Design of graph filters and filterbanks
def graph_filter(U, A, filter_type):
    
    if filter_type == 'low_pass':
        h = [1 if idx<=int((A.shape[0]-1)/2) else 0 for idx in range(0, A.shape[0])]

    elif filter_type == 'high_pass':
        h = [1 if idx>=int((A.shape[0]-1)/2)else 0 for idx in range(0, A.shape[0])]

    I = np.identity(A.shape[0])
    h_lambda = np.diag(np.matmul(h, I))
    H = np.matmul(np.matmul(U, h_lambda),np.linalg.inv(U))
    return H

In [523]:
def classification_results(results_dict):
    for modality in ['t2f', 't1w', 't1ce', 't2w']:
        results_dict[modality]= {}
        for experiment_number in range(1,26):
            results_dict[modality][experiment_number] = {}
            df = load_relevant_features(modality, experiment_number)
            codel_labels = df['1p_19q_co_del_status']
            idh_labels = df['IDH1']
            idh_labels = np.array(idh_labels)
            df.drop(columns=['IDH1', '1p_19q_co_del_status'], inplace=True)

            drop_columns, correlated_pair = get_features_to_exclude(df)
            filtered_df = drop_correlated_features(df, drop_columns)

            M = form_minimum_spanning_tree(filtered_df)
            A = np.abs(nx.adj_matrix(M)).todense()
            if A.shape[0] < 5:
                print('Experiment number {} not considered'.format(experiment_number))
                continue

            G = pygsp.graphs.Graph(A)
            G.compute_fourier_basis()
            U, _ = graph_fourier_bases(G)

            X = normalize_signal(filtered_df)
            X_normalized_graph = compute_graph_gft(U, X.T)
            total_variation = compute_graph_tv(G, X)  
            
            X_tv = np.hstack((X, np.reshape(total_variation, (total_variation.shape[0], 1))))
            H = graph_filter(U, A, filter_type='low_pass') 
            # Fourier domain signal after filtering
            Y_normalized_graph = np.matmul(np.matmul(H, np.linalg.inv(U)), X.T)

            # Inverse GFT of the filtered signal
            Y = np.matmul(U, Y_normalized_graph)
            tv_filtered = compute_graph_tv(G, Y.T)
            #Y_normalized_graph_tv = np.hstack((Y_normalized_graph.T, np.reshape(tv_filtered, (tv_filtered.shape[0], 1))))
            #X_normalized_graph_tv = np.hstack((X_normalized_graph, np.reshape(total_variation, (total_variation.shape[0], 1))))
            Y_tv = np.hstack((Y.T, np.reshape(tv_filtered, (tv_filtered.shape[0], 1))))
            
            X, idh_labels = shuffle(Y_tv, idh_labels, random_state=2019)

            train_accuracy = []
            test_accuracy = []
            test_precision = []
            test_recall = []
            test_auc = []
            train_auc = []
            for index, (train, test) in enumerate(kfold.split(X, idh_labels)):
                classifier.fit(X[train], idh_labels[train])
                train_accuracy.append(accuracy_score(idh_labels[train], classifier.predict(X[train])))
                test_accuracy.append(accuracy_score(idh_labels[test], classifier.predict(X[test])))
                test_recall.append(recall_score(idh_labels[test], classifier.predict(X[test])))
                test_precision.append(precision_score(idh_labels[test], classifier.predict(X[test])))
                train_auc.append(roc_auc_score(idh_labels[train], classifier.predict(X[train])))
                test_auc.append(roc_auc_score(idh_labels[test], classifier.predict(X[test])))

            results_dict[modality][experiment_number]['features'] = list(filtered_df.columns)
            results_dict[modality][experiment_number]['aucroc'] = [np.mean(np.array(test_auc)), 
                                                                  np.std(np.array(test_auc))]
            results_dict[modality][experiment_number]['accuracy'] = [np.mean(np.array(test_accuracy)), 
                                                                  np.std(np.array(test_accuracy))]
            results_dict[modality][experiment_number]['precision'] = [np.mean(np.array(test_precision)), 
                                                                  np.std(np.array(test_precision))]
            results_dict[modality][experiment_number]['recall'] = [np.mean(np.array(test_recall)), 
                                                                   np.std(np.array(test_recall))]
        
    return results_dict

In [524]:
# vertex_domain = {} #a
#vertex_domain_filtered = {} #b
#vertex_domain_with_tv = {} #c
#fourier_domain = {} #d
#filtered_fourier_domain = {} #e
#filtered_fourier_domain_with_tv = {} #f
#fourier_domain_with_tv = {} #g
vertex_domain_filtered_with_tv = {} #Extra

In [525]:
# vertex_domain = classification_results(vertex_domain)
# vertex_domain_filtered = classification_results(vertex_domain_filtered)
#vertex_domain_with_tv = classification_results(vertex_domain_with_tv)
#fourier_domain = classification_results(fourier_domain)
#filtered_fourier_domain = classification_results(filtered_fourier_domain)
#filtered_fourier_domain_with_tv = classification_results(filtered_fourier_domain)
#fourier_domain_with_tv = classification_results(fourier_domain_with_tv)
#vertex_domain_filtered_with_tv = classification_results(vertex_domain_filtered_with_tv)

/home/raghuram/Desktop/radiomics_venv/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/raghuram/Desktop/radiomics_venv/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/raghuram/Desktop/radiomics_venv/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/raghuram/Desktop/radiomics_venv/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/raghuram/Desktop/radiomics_venv/lib/python3.7/site-packages/sklearn/linear_mod

Experiment number 3 not considered


/home/raghuram/Desktop/radiomics_venv/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/raghuram/Desktop/radiomics_venv/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/raghuram/Desktop/radiomics_venv/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/raghuram/Desktop/radiomics_venv/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/raghuram/Desktop/radiomics_venv/lib/python3.7/site-packages/sklearn/linear_mod

Experiment number 8 not considered


/home/raghuram/Desktop/radiomics_venv/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/raghuram/Desktop/radiomics_venv/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/raghuram/Desktop/radiomics_venv/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/raghuram/Desktop/radiomics_venv/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/raghuram/Desktop/radiomics_venv/lib/python3.7/site-packages/sklearn/linear_mod

...

Experiment number 9 not considered
Experiment number 10 not considered
Experiment number 11 not considered
Experiment number 12 not considered
Experiment number 13 not considered
Experiment number 14 not considered


/home/raghuram/Desktop/radiomics_venv/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/raghuram/Desktop/radiomics_venv/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/raghuram/Desktop/radiomics_venv/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Experiment number 16 not considered
Experiment number 17 not considered
Experiment number 18 not considered
Experiment number 19 not considered
Experiment number 20 not considered


/home/raghuram/Desktop/radiomics_venv/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/raghuram/Desktop/radiomics_venv/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/raghuram/Desktop/radiomics_venv/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/raghuram/Desktop/radiomics_venv/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/raghuram/Desktop/radiomics_venv/lib/python3.7/site-packages/sklearn/linear_mod

Experiment number 25 not considered
